In [ ]:
import scipy.signal
import sys
import torch
import torch.nn as nn
import sys
sys.path.append("/home/xhnfly/Cosmic_rays_X/XRL/")
import numpy as np

In [ ]:
import math
import random
from copy import deepcopy
from this import d
import torch
from torch.optim import Adam
from torch.optim import RMSprop
import XRL_main
import gym
import time
from collections import namedtuple, deque
import neptune.new as neptune
import datetime

# from tqdm import tqdm

In [ ]:
nep_log = neptune.init(
    project="xhnfirst/DDPG-mian-test1",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1NTg5MDI2OS01MTVmLTQ2YjUtODA1Yy02ZWQyNDgxZDcwN2UifQ==",
)

In [ ]:
import math
import sys
from time import strftime
from time import gmtime

class view_bar:
    def __init__(self, message, num, total, value_name, value_data, start_time_rec, r_true):
        self.message = message
        self.num = num
        self.total = total
        self.r_true = r_true
        self.message2 = "True time of r = "
        self.message3 = "time cost = "
        rate = self.num  / self.total
        rate_num = int(rate * 40)
        rate_nums = math.ceil(rate * 100)
        time_rec = datetime.datetime.now()
        execution_time = time_rec - start_time_rec


        r = '\r%s:[%s%s]%d%%\t%d/%d\t%s%f\t%s%s\t%s%s' % (self.message, "=" * rate_num,
                                        " " * (40 - rate_num), rate_nums, self.num , self.total, value_name, value_data, self.message3, execution_time, self.message2, str(self.r_true))

        sys.stdout.write(r)
        sys.stdout.flush()

# a =10
# for i in range(60000):
#     view_bar("epoch ", i+1, 60000, 'a = ', float(a))
#     a += 0.01


In [ ]:
env = gym.make('Reacher-v2')
test_env = gym.make('Reacher-v2')
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device = ', device)

In [ ]:
def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


class MLPActor(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation, act_limit):
        super().__init__()
        pi_sizes = [obs_dim] + list(hidden_sizes) + [act_dim]
        self.pi = mlp(pi_sizes, activation, nn.Tanh)
        self.act_limit = act_limit

    def forward(self, obs):
        # Return output from network scaled to action space limits.
        return self.act_limit * self.pi(obs)

class MLPQFunction(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.q = mlp([obs_dim + act_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs, act):
        q = self.q(torch.cat([obs, act], dim=-1))
        return torch.squeeze(q, -1) # Critical to ensure q has right shape.

class MLPActorCritic(nn.Module):

    def __init__(self, observation_space, action_space, hidden_sizes=(256,256),
                 activation=nn.ReLU, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super().__init__()

        obs_dim = observation_space.shape[0]
        act_dim = action_space.shape[0]
        act_limit = action_space.high[0]

        # build policy and value functions
        self.pi = MLPActor(obs_dim, act_dim, hidden_sizes, activation, act_limit).to(device)
        self.q = MLPQFunction(obs_dim, act_dim, hidden_sizes, activation).to(device)

    def act(self, obs):
        with torch.no_grad():
            return self.pi(obs)

In [ ]:
Transition = namedtuple('Transition',
                        ('obs', 'act', 'rew', 'next_obs', 'done'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [8]:

params = {
    "batch_size": 128,
    "dropout": 0.2,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "hid": 64,
    "l": 3,
    "seed": 0,
    "steps_per_epoch": 3000,
    "epochs": 100,
    "replay_size": int(1e6),
    "gamma": 0.99,
    "polyak": 0.995,
    "pi_lr": 1e-4,
    "q_lr": 1e-4,
    "batch_size": 500,
    "start_steps": 10000, 
    "update_after": 1000,
    "update_every": 50,
    "act_noise": 0.01,
    "num_test_episodes": 10,
    "max_ep_len": 1000
}

nep_log["parameters"] = params


ac_kwargs=dict(hidden_sizes=[params["hid"]]*params["l"])

torch.manual_seed(params["seed"])
np.random.seed(params["seed"])
obs_dim = env.observation_space.shape[0]

print('obs_dim = ', obs_dim)
act_dim = env.action_space.shape[0]
print('act_dim = ', act_dim)
# Action limit for clamping: critically, assumes all dimensions share the same bound!
act_limit = env.action_space.high[0]
print('act_limit = ', act_limit)
# Create actor-critic module and target networks
ac = MLPActorCritic(env.observation_space, env.action_space, **ac_kwargs)
ac_targ = deepcopy(ac)

# Freeze target networks with respect to optimizers (only update via polyak averaging)
for p in ac_targ.parameters():
    p.requires_grad = False

memory = ReplayMemory(params["replay_size"])


# Set up function for computing DDPG Q-loss
def compute_loss_q(data):

    o = torch.cat(data.obs).float()
    a = torch.cat(data.act).float()
    r = torch.cat(data.rew).float()
    o2 =torch.cat(data.next_obs).float()
    d = torch.cat(data.done).float()

    q = ac.q(o,a)


    # Bellman backup for Q function
    with torch.no_grad():
        q_pi_targ = ac_targ.q(o2, ac_targ.pi(o2))
        backup = r + params["gamma"] * (1 - d) * q_pi_targ

    # MSE loss against Bellman backup
    loss_q = ((q - backup)**2).mean()

    return loss_q

# Set up function for computing DDPG pi loss
def compute_loss_pi(data):

    o = torch.cat(data.obs).float()

    q_pi = ac.q(o, ac.pi(o))

    return -q_pi.mean()

pi_optimizer = RMSprop(ac.pi.parameters(), lr=params["pi_lr"])
q_optimizer = RMSprop(ac.q.parameters(), lr=params["q_lr"])

def update(data):
    # First run one gradient descent step for Q.


    q_optimizer.zero_grad()
    loss_q = compute_loss_q(data)

    loss_q.backward()

    q_optimizer.step()


    # Freeze Q-network so you don't waste computational effort 
    # computing gradients for it during the policy learning step.
    for p in ac.q.parameters():
        p.requires_grad = False

    # Next run one gradient descent step for pi.
    pi_optimizer.zero_grad()
    loss_pi = compute_loss_pi(data)
    loss_pi.backward()
    pi_optimizer.step()

    # Unfreeze Q-network so you can optimize it at next DDPG step.
    for p in ac.q.parameters():
        p.requires_grad = True


    # Finally, update target networks by polyak averaging.
    with torch.no_grad():
        for p, p_targ in zip(ac.parameters(), ac_targ.parameters()):
            # NB: We use an in-place operations "mul_", "add_" to update target
            # params, as opposed to "mul" and "add", which would make new tensors.
            p_targ.data.mul_(params["polyak"])
            p_targ.data.add_((1 - params["polyak"]) * p.data)


def get_action(o, noise_scale):
    a = ac.act(torch.tensor(o, dtype=torch.float32, device=device))
    a += noise_scale * torch.randn(act_dim).to(device)
    return torch.clip(a, -act_limit, act_limit)

def test_agent():
    for j in range(params["num_test_episodes"]):
        o, d, ep_ret, ep_len = test_env.reset(), False, 0, 0
        while not(d or (ep_len == params["max_ep_len"])):
            # Take deterministic actions at test time (noise_scale=0)
            a_cpu = get_action(o, 0).cpu().data.numpy()
            # a = get_action(o, 0)
            o, r, d, _ = test_env.step(a_cpu)
            if d == True:
                r += 10
            else:
                r += -1
            ep_ret += r
            ep_len += 1
        nep_log["test/reward"].log(ep_ret/ep_len)






# Prepare for interaction with environment
total_steps = params["steps_per_epoch"] * params["epochs"]
start_time = time.time()
o, ep_ret, ep_len = env.reset(), 0, 0
o = torch.tensor([o], device=device)

start_time_rec = datetime.datetime.now()
# Main loop: collect experience in env and update/log each epoch
r_true = 0
for t in range(total_steps):
    
    # Until start_steps have elapsed, randomly sample actions
    # from a uniform distribution for better exploration. Afterwards, 
    # use the learned policy (with some noise, via act_noise). 
    if t > params["start_steps"]:
        a = get_action(o, params["act_noise"])
        # a_cpu = a
        a_cpu = a.cpu().data.numpy()
        
    else:
        a = env.action_space.sample()
        a_cpu = a
        a = torch.tensor([a], device=device)

    # Step the env
    o2, r, d, _ = env.step(a_cpu)
    print('r = ', r)
    print('d = ', d)
    if d == True:
        r += 10
        r_true += 1
    else:
        r += -1
    ep_ret += r
    ep_len += 1
    ep_ret_main = ep_ret/ep_len


    # Ignore the "done" signal if it comes from hitting the time
    # horizon (that is, when it's an artificial terminal signal
    # that isn't based on the agent's state)
    d = False if ep_len==params["max_ep_len"] else d

    # a = torch.tensor([a], device=device)
    o2 = torch.tensor([o2], device=device)
    r = torch.tensor([r], device=device)
    d = torch.tensor([d], device=device)

    # Store experience to replay buffer
    memory.push(o, a, r, o2, d)
    # Super critical, easy to overlook step: make sure to update 
    # most recent observation!
    o=o2

    # End of trajectory handling
    if d or (ep_len == params["max_ep_len"]):
        o, ep_ret, ep_len = env.reset(), 0, 0
        o = torch.tensor([o], device=device)


    # Update handling
    if t >= params["update_after"] and t % params["update_every"] == 0:
        view_bar("steps: ", t, total_steps, 'main of rewards = ', float(ep_ret_main), start_time_rec, r_true)
        
        nep_log["train/reward"].log(ep_ret_main)
        for i in range(params["update_every"]):

            transitions = memory.sample(params["batch_size"])
            # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
            # detailed explanation). This converts batch-array of Transitions
            # to Transition of batch-arrays.
            batch = Transition(*zip(*transitions))

            # print('batch = ', str(batch))
            update(data=batch)

    # End of epoch handling
    if (t+1) % params["steps_per_epoch"] == 0:
        epoch = (t+1) // params["steps_per_epoch"]

        # Test the performance of the deterministic version of the agent.
        test_agent()



r =  -1.668342245574622
d =  False
r =  -1.8526876968828192
d =  False
r =  -1.6597030317270076
d =  False
r =  -1.5049055332145436
d =  False
r =  -1.6089879745124633
d =  False
r =  -1.597133125252244
d =  False
r =  -1.4091722368505915
d =  False
r =  -1.1871197735891004
d =  False
r =  -1.118426095021267
d =  False
r =  -1.1456230932961722
d =  False
r =  -1.072164134394387
d =  False
r =  -1.2064633221464078
d =  False
r =  -0.5984520495110661
d =  False
r =  -0.717556463480573
d =  False
r =  -0.5356604237995711
d =  False
r =  -0.4008689891222197
d =  False
r =  -0.5561161899156417
d =  False
r =  -0.864772222488631
d =  False
r =  -0.9270559318115437
d =  False
r =  -0.7695818091938607
d =  False
r =  -0.5316400806323962
d =  False
r =  -0.3975056154483827
d =  False
r =  -0.7166902857770469
d =  False
r =  -0.9134690231433539
d =  False
r =  -0.8036148134971237
d =  False
r =  -0.5127427644185356
d =  False
r =  -0.38565974180064044
d =  False
r =  -0.7319629256977036
d =  Fal

KeyboardInterrupt: 

In [ ]:
nep_log.stop()